<a href="https://colab.research.google.com/github/Squirtle007/CUDA-Q/blob/main/colab/CUDA-QX/optimizations/cudaq_knapsack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cudaq-solvers==0.2.1 pyqubo==1.5.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 89.1 

# **Solving Knapsack Problem with CUDA-Q Solvers**

The **Knapsack Problem** is a combinatorial optimization problem where the objective is to maximize the total value of selected items, subject to a weight constraint. The objective function $ f(x) $ in **QUBO** (Quadratic Unconstrained Binary Optimization) formulation to be minimized:

$$
f(x) = -\sum (v_i \times x_i) + \lambda \left( \sum (w_i \times x_i) - W \right)^2
$$

The goal is to maximize the total value while penalizing weight violations. Below is a detailed breakdown of the QUBO:

## **1. Item Data**  
Items have associated weights and values, represented by lists `weights` and `values`.

## **2. Binary Variables**  
Each item is assigned a binary variable (`items`), where 1 indicates the item is selected, and 0 indicates it is not.

## **3. Total Weight and Value**  
- Total weight: $ \sum (w_i \times x_i) $  
- Total value: $ \sum (v_i \times x_i) $  

where $ x_i $ is the binary selection variable for item $ i $.

## **4. Weight Constraint**
A penalty is applied if the total weight exceeds the maximum allowable weight $ W $, represented by:

$$
\left( \sum (w_i \times x_i) - W \right)^2
$$
where $ \lambda $ is the penalty scale factor of the penalty for the constraint.

In [2]:
from pyqubo import Binary, Constraint


# Item data (weights and values)
weights = [3, 5, 2]
values = [8, 16, 9]
max_weight = 7

# Binary variables for item selection
variables = [Binary(f'item_{i}') for i in range(3)]

# Total weight and value calculations
total_weight = sum(w * x for w, x in zip(weights, variables))
total_value = sum(v * x for v, x in zip(values, variables))

# Objective: minimize (negative value + weight constraint penalty)
penalty = 5
weight_constraint = Constraint((total_weight - max_weight)**2, label="Weight_Constraint")
objective = -total_value + penalty * weight_constraint

# Compile
model = objective.compile()

Note that the QUBO formulation from PyQUBO is based on the 0 and 1 basis. Here, we aim to transform it into the -1 and 1 basis and construct the Hamiltonian with Ising formulation, making it compatible for use with CUDA-Q:

In [3]:
import cudaq
from cudaq import spin
import numpy as np
from typing import List


def to_cudaq(model):
    """
    Convert a binary optimization model to QAOA inputs compatible with CUDA-Q.

    This function extracts the linear and quadratic coefficients from the model's
    Ising formulation, maps variables to qubit indices, and constructs CUDA-Q Hamiltonian.
    """

    h, J, offset = model.to_ising()

    # Map variables to qubit indices
    vars_set = set(h) | {v for pair in J for v in pair}
    var_to_index = {var: idx for idx, var in enumerate(sorted(vars_set))}

    # Initialization
    H = 0

    # Fill in linear terms (diagonal)
    for var, weight in h.items():
        idx = var_to_index[var]
        H += weight * spin.z(idx)

    # Fill in quadratic terms (off-diagonal)
    for (v1, v2), weight in J.items():
        i, j = var_to_index[v1], var_to_index[v2]
        H += weight * spin.z(i) * spin.z(j)

    H += offset

    return H


H = to_cudaq(model)
print("Hamiltonian:", H)

Hamiltonian: [25+0j] IZZ
[15+0j] ZIZ
[-58+0j] IZI
[37.5+0j] ZZI
[-34+0j] ZII
[-24.5+0j] IIZ
[51+0j] III



##**QAOA using cudaq-solvers.qaoa**
The **Quantum Approximate Optimization Algorithm (QAOA)** is a variational quantum algorithm designed to solve **combinatorial optimization problems  (COPs)**.
CUDA-Q Solvers provides the built-in `cudaq-solvers.qaoa` API, offering a simple and intuitive approach to tackle COPs using QAOA:

In [4]:
import cudaq_solvers as solvers
import numpy as np


# Get the number of parameters we'll need
num_layers = 3
parameter_count = solvers.get_num_qaoa_parameters(H,
                                                  num_layers,
                                                  full_parameterization=True)

# Create the initial parameters to begin optimization
initial_parameters = np.random.uniform(-np.pi / 8, np.pi / 8, parameter_count)

# Set up the optimizer for convergence; supported cudaq-x optimizers include 'cobyla' and 'lbfgs'
optimizer = 'cobyla'

# Run QAOA, specify full parameterization using an optimization parameter for
# every term in the problem Hamiltonian and the mixer hamiltonian.
opt_value, opt_params, opt_config = solvers.qaoa(H,
                                                 num_layers,
                                                 initial_parameters,
                                                 optimizer=optimizer,
                                                 full_parameterization=True,
                                                 )

# Print the results
print('Optimal energy: ', opt_value)
print('Sampled states: ', opt_config)
print('Optimal Configuration: ', opt_config.most_probable())

Optimal energy:  -23.143728584922556
Sampled states:  { 000:1 010:29 100:963 110:7 }

Optimal Configuration:  100


Note that in **PyQUBO**, the variables (`Binary`) are originally **0 or 1**:  $ x_i \in \{0, 1\} $

Meanwhile, in the **Ising form**, the variables are represented as **-1 or +1**: $ s_i \in \{-1, 1\} $

To convert the sampled bitstring back to the pre-defined variable $x$, we should apply the transformation:
$$
x = \frac{1 - s}{2}
$$

In [5]:
def to_variables(variables, most_probable_bits):
    """
    Convert Ising bitstring string (s ∈ {-1,1}) to predefined Binary variables (x ∈ {0,1}).
    """
    best_result = {str(variables[i]): int((1 - (1 if bit == '1' else -1)) // 2) for i, bit in enumerate(most_probable_bits)}
    return best_result


# Print result
most_probable_bits = opt_config.most_probable()
best_result = to_variables(variables, most_probable_bits)
print("Best result:", best_result)

Best result: {"Binary('item_0')": 0, "Binary('item_1')": 1, "Binary('item_2')": 1}


##**Comparison with Simulated Annealing solver from `neal`**

In [6]:
import neal


# Convert the pre-defined model to QUBO
qubo, offset = model.to_qubo()

# Solve with simulated annealing
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(qubo, num_reads=100)

# Decode solution
decoded_samples = model.decode_sampleset(sampleset)
best = min(decoded_samples, key=lambda s: s.energy)

# Print result
print("Best result:", best.sample)
print("Energy (including offset):", best.energy)

Best result: {'item_2': 1, 'item_1': 1, 'item_0': 0}
Energy (including offset): -25.0
